<a href="https://colab.research.google.com/github/iBibek/dsci6015-lab1/blob/main/Lab_3_1_Phish_and_Spam_Classification_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
phishing_dataset = np.genfromtxt('/content/drive/MyDrive/6015/phishing_dataset.csv', delimiter=',', dtype=np.int32)
samples = phishing_dataset[:,:-1]
targets = phishing_dataset[:, -1]

In [5]:
from sklearn.model_selection import train_test_split

training_samples, testing_samples, training_targets, testing_targets = train_test_split(samples, targets, test_size=0.2, random_state=42)


In [150]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [152]:
log_classifier = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-4, shuffle=True,loss='log'))
log_classifier.fit(training_samples, training_targets )
predictions = log_classifier.predict(testing_samples)
accuracy = 100.0 * metrics.accuracy_score(testing_targets, predictions)
print ("Logistic Regression accuracy: " + str(accuracy))

Logistic Regression accuracy: 92.4468566259611


In [ ]:
# USING THE PYTORCH FOR the linear and logistic regression

In [ ]:
import torch
class Logistic_Reg_model(torch.nn.Module):
 def __init__(self,no_input_features):
   super(Logistic_Reg_model,self).__init__()
   self.layer1=torch.nn.Linear(no_input_features,20)
   self.layer2=torch.nn.Linear(20,1)
 def forward(self,x):
   y_predicted=self.layer1(x)
   y_predicted=torch.sigmoid(self.layer2(y_predicted))
   return y_predicted

In [ ]:
model=Logistic_Reg_model(30)

In [ ]:
model

Logistic_Reg_model(
  (layer1): Linear(in_features=30, out_features=20, bias=True)
  (layer2): Linear(in_features=20, out_features=1, bias=True)
)

In [ ]:
criterion=torch.nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [108]:
import numpy as np
import torch
x_train=torch.from_numpy(training_samples.astype(np.float32))
x_test=torch.from_numpy(testing_samples.astype(np.float32))
y_train=torch.from_numpy(training_targets.astype(np.float32))
y_test=torch.from_numpy(testing_targets.astype(np.float32))

In [56]:
import torch

from torch.autograd import Variable

class LinearRegression(torch.nn.Module):   
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(30, 1)    
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

model = LinearRegression()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(500):
    model.train()
    optimizer.zero_grad()    # Forward pass
    y_pred = model(x_train)    # Compute Loss
    loss = criterion(y_pred.reshape(-1), y_train)    # Backward pass
    loss.backward()
    optimizer.step()
    
    
    

In [58]:
model.eval()
tar_pred = model(x_test)
tr=np.array(y_test.cpu().detach().numpy() ) == 1
op=np.array(tar_pred.cpu().detach().numpy()) >=0.5
print(metrics.accuracy_score(tr,op))
    

0.6897331524197196


LOGISTIC REGRESSION

In [145]:
from torch.nn import functional as F
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(30, 1)     
    def forward(self, x):
        y_pred = F.sigmoid(self.linear(x))
        return y_pred
modelR = LogisticRegression()

# we need to change the faetures to 0 and 1 only
y_train=[ 0 if x<=0 else 1 for x in training_targets ]
y_train=torch.from_numpy(np.array(y_train).astype(np.float32))
# use the  Binary Cross Entropy for loss since we are predicting Binary Value as op
criterion = torch.nn.BCELoss(size_average=True)
for epoch in range(500):
    modelR.train()
    optimizer.zero_grad()    
    y_pred = modelR(x_train)    
    #use the binary cross entropy loss with the logits,
    loss=torch.nn.functional.binary_cross_entropy_with_logits(y_pred.reshape(-1), y_train)
    loss.backward()
    optimizer.step()




/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [146]:
modelR.eval()
tar_pred = modelR(x_test)
tr=np.array(y_test.cpu().detach().numpy() ) == 1
op=np.array(tar_pred.cpu().detach().numpy()) >=0.5
print("The accuracy for Logistic Regression Model is: ", 100* metrics.accuracy_score(tr,op))

The accuracy for Logistic Regression Model is:  70.5563093622795
